In [1]:
%pip install pandas

import gc
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
input_parquet_file_path = '../data/interm/2.1_taxdata.parquet'

output_parquet_file_path = '../data/interm/2.2_taxdata.parquet'

In [3]:
data_df = pd.read_parquet(input_parquet_file_path)

code_insee_identifier = 'Clean code INSEE'

In [4]:
# Fill in missing values in the libelle commune field
missing_libelle_names = {
    "97102": "ANSE-BERTRAND",
    "97104": "BAILLIF",
    "97105": "BASSE-TERRE",
    "97106": "BOUILLANTAIS",
    "97107": "CAPESTERRE-BELLE-EAU",
}
data_df['Libellé commune'] = data_df[code_insee_identifier].map(missing_libelle_names).fillna(data_df['Libellé commune'])

# Replace VILLE DE PARIS with PARIS all over the places
data_df['Libellé commune'] = data_df['Libellé commune'].replace('VILLE DE PARIS', 'PARIS')


# After the Clean code INSEE calculations, add a LABEL to represent the Clean code INSEE - Commune name - Department name
descriptive_label_df = pd.DataFrame({
    'LABEL': data_df[code_insee_identifier] + ' - ' + data_df['Libellé commune'] + ' - ' + data_df["DEPARTEMENT_LABEL"]
})

result_df = pd.concat([data_df, descriptive_label_df], axis=1)

# Cleanup communes that don't have more then 5 entries or data for less than 5 years

In [5]:
min_number_of_entries = 5
min_number_of_years = 5

# Filter the dataset to keep only relevant columns: Clean code insee and annee
df_filtered = result_df[[code_insee_identifier, 'ANNEE']]

# Group by 'Clean code INSEE' and filter for communes with at least 10 entries and data for at least 5 years
commune_counts = df_filtered.groupby('Clean code INSEE').agg({
    'ANNEE': ['count', 'nunique']
})
commune_counts.columns = ['entry_count', 'year_count']

# Get communes that do not fit the criteria
communes_to_drop = commune_counts[(commune_counts['entry_count'] < min_number_of_entries) | (commune_counts['year_count'] < min_number_of_years)].index

# Filter the original dataframe to get the data for these communes
df_not_fit = result_df[result_df[code_insee_identifier].isin(communes_to_drop)]
df_not_fit_codes_labels = df_not_fit[[code_insee_identifier, 'LABEL']]

# Filter the original dataframe to get the data for communes that fit the criteria
cleaned_data_df = result_df[~result_df[code_insee_identifier].isin(communes_to_drop)]

In [6]:

print('The following communes are be dropped from the calculations: ', communes_to_drop)

# Merge with commune_counts to get entry count and year count
df_not_fit_codes_labels = df_not_fit_codes_labels.merge(commune_counts, left_on=code_insee_identifier, right_index=True)
df_not_fit_codes_labels = df_not_fit_codes_labels.drop_duplicates(subset=[code_insee_identifier, 'LABEL'])

df_not_fit_codes_labels.head(25)

The following communes are be dropped from the calculations:  Index(['07046', '08221', '13120', '66200', '67211', '72206', '77535', '77536',
       '77537', '78689', '80772', '91693', '91694', '95691'],
      dtype='string', name='Clean code INSEE')


,Clean code INSEE,LABEL,entry_count,year_count
364025,72206,72206 - MONTFORT LE ROTROU - Sarthe,4,4
375182,13120,13120 - RIVES DE L'ETANG DE BERRE - Bouches-du...,3,3
402639,77535,77535 - MARNE-LA-VALLEE - Seine-et-Marne,3,3
402640,77536,77536 - GRAND-MELUN - Seine-et-Marne,3,3
402641,77537,77537 - SENART-VILLENEUVE - Seine-et-Marne,3,3
402902,78689,78689 - ST QUENTIN EN YVELINES - Yvelines,3,3
403932,80772,80772 - VACQUERIE - Somme,3,3
406842,91693,91693 - EVRY VILLE NOUVELLE - Essonne,3,3
406843,91694,91694 - ROUGEAU-SENART - Essonne,3,3
407152,95691,95691 - CERGY-POINTOISE - Val-d'Oise,3,3


In [7]:
# Save the processed data back to a Parquet file
cleaned_data_df.to_parquet(output_parquet_file_path)

print(f'Number of rows: {cleaned_data_df.shape[0]}')
num_unique_clean_code_insee = cleaned_data_df[code_insee_identifier].nunique()

print(f'Number of communes: {num_unique_clean_code_insee}')

del result_df
del data_df
gc.collect()

Number of rows: 1528887
Number of communes: 36746


0